In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:

# Hyper-parameters 
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:

# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=False)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [10]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [11]:


# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [6]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)


In [7]:
# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [8]:

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #如果原始图像数据的形状是(batch_size, height, width, channels)
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)
        # 这是因为在该代码段中，图像数据被展平成一维向量，以作为全连接神经网络的输入。
        #通常情况下，全连接神经网络的输入是一维的，因此需要将图像数据从二维或三维的形状转换为一维。
        # 具体而言，reshape操作通过指定维度的大小来调整数据的形状，其中-1表示自动计算维度大小。
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2390
Epoch [1/5], Step [200/600], Loss: 2.1325
Epoch [1/5], Step [300/600], Loss: 2.0604
Epoch [1/5], Step [400/600], Loss: 1.9342
Epoch [1/5], Step [500/600], Loss: 1.8954
Epoch [1/5], Step [600/600], Loss: 1.7690
Epoch [2/5], Step [100/600], Loss: 1.6977
Epoch [2/5], Step [200/600], Loss: 1.6617
Epoch [2/5], Step [300/600], Loss: 1.6658
Epoch [2/5], Step [400/600], Loss: 1.5475
Epoch [2/5], Step [500/600], Loss: 1.5920
Epoch [2/5], Step [600/600], Loss: 1.4572
Epoch [3/5], Step [100/600], Loss: 1.3458
Epoch [3/5], Step [200/600], Loss: 1.3498
Epoch [3/5], Step [300/600], Loss: 1.2691
Epoch [3/5], Step [400/600], Loss: 1.4172
Epoch [3/5], Step [500/600], Loss: 1.2959
Epoch [3/5], Step [600/600], Loss: 1.1866
Epoch [4/5], Step [100/600], Loss: 1.2654
Epoch [4/5], Step [200/600], Loss: 1.1418
Epoch [4/5], Step [300/600], Loss: 1.1669
Epoch [4/5], Step [400/600], Loss: 1.2261
Epoch [4/5], Step [500/600], Loss: 1.2061
Epoch [4/5], Step [600/600], Loss:

In [9]:


# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        #PyTorch中，dim=0 表示沿着列的方向计算最大值，而 dim=1 表示沿着行的方向计算最大值     
        #返回两个值大值和索引
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))



Accuracy of the model on the 10000 test images: 83.05999755859375 %


In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')